In [74]:
from pyfaidx import Fasta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Full list of the transcripts

Não faz sentido ir buscar o gene id, 

In [75]:
petides = Fasta('Homo_sapiens.GRCh38.pep.all.fa')

petide_sequence = pd.DataFrame(petides.keys(), columns=["id_version"])
petide_sequence["Protein_ID"]= petide_sequence["id_version"].str.split(".").str.get(0) #

In [76]:
def get_protein_seq(id):
    seq=petides[id][:].seq
    return seq


In [77]:
petide_sequence["protein_seq"]=petide_sequence["id_version"].apply(get_protein_seq)
petide_sequence

,id_version,Protein_ID,protein_seq
0,ENSP00000451042.1,ENSP00000451042,EI
1,ENSP00000452494.1,ENSP00000452494,TGGY
2,ENSP00000451515.1,ENSP00000451515,PSY
3,ENSP00000488240.1,ENSP00000488240,GTGG
4,ENSP00000487941.1,ENSP00000487941,GTGG
...,...,...,...
122609,ENSP00000448878.1,ENSP00000448878,XEITSSPQGLDNPALLRDMLLANPHELSLLKERNPPLAEALLSGDL...
122610,ENSP00000341963.1,ENSP00000341963,MSSLPTSDGFNHPARSSGQSPDVGNPMSLARSVSASVCPIKPSDSD...
122611,ENSP00000364956.3,ENSP00000364956,MEPGEVKDRILENISLSVKKLQSYFAACEDEIPAIRNHDKVLQRLC...
122612,ENSP00000364950.2,ENSP00000364950,MEPGEVKDRILENISLSVKKLQSYFAACEDEIPAIRNHDKVLQRLC...


# Get the Canonical dataset

In [78]:
from pybiomart import Dataset
dataset = Dataset(name='hsapiens_gene_ensembl',
           host='http://www.ensembl.org')

dataset_df=dataset.list_attributes()
dataset_df.to_csv("pybiomart_dataset.csv")


In [79]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')


df = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name', 'ensembl_peptide_id', 'transcript_is_canonical'])

In [80]:
pedtide_df=df[df["Ensembl Canonical"]==1].copy() # filter to only show the canonical genes

In [81]:
pedtide_df

,Gene stable ID,Gene name,Protein stable ID,Ensembl Canonical
0,ENSG00000210049,MT-TF,NaN,1.0
1,ENSG00000211459,MT-RNR1,NaN,1.0
2,ENSG00000210077,MT-TV,NaN,1.0
3,ENSG00000210082,MT-RNR2,NaN,1.0
4,ENSG00000209082,MT-TL1,NaN,1.0
...,...,...,...,...
191666,ENSG00000288629,NaN,ENSP00000502602,1.0
191667,ENSG00000288678,NaN,ENSP00000505765,1.0
191669,ENSG00000290825,DDX11L2,NaN,1.0
191670,ENSG00000227232,WASH7P,NaN,1.0


In [82]:
print(pedtide_df.isna().sum())

Gene stable ID           0
Gene name            22601
Protein stable ID    46891
Ensembl Canonical        0
dtype: int64


### Merge the full transcripts with the canonical transcripts

The purpose of this is to extract the Transcript version

In [83]:
cano_df = pd.merge(pedtide_df,petide_sequence , left_on='Protein stable ID', right_on='Protein_ID')

In [84]:
clean_df = cano_df[["Gene name","Gene stable ID","Protein_ID","protein_seq"]].copy()

In [85]:
clean_df

,Gene name,Gene stable ID,Protein_ID,protein_seq
0,MT-ND1,ENSG00000198888,ENSP00000354687,MPMANLLLLIVPILIAMAFLMLTERKILGYMQLRKGPNVVGPYGLL...
1,MT-ND2,ENSG00000198763,ENSP00000355046,MNPLAQPVIYSTIFAGTLITALSSHWFFTWVGLEMNMLAFIPVLTK...
2,MT-CO1,ENSG00000198804,ENSP00000354499,MFADRWLFSTNHKDIGTLYLLFGAWAGVLGTALSLLIRAELGQPGN...
3,MT-CO2,ENSG00000198712,ENSP00000354876,MAHAAQVGLQDATSPIMEELITFHDHALMIIFLICFLVLYALFLTL...
4,MT-ATP8,ENSG00000228253,ENSP00000355265,MPQLNTTVWPTMITPMLLTLFLITQLKMLNTNYHLPPSPKPMKMKN...
...,...,...,...,...
23776,CNKSR1,ENSG00000142675,ENSP00000354609,MEPVETWTPGKVATWLRGLDDSLQDYPFEDWQLPGKNLLQLCPQSL...
23777,NaN,ENSG00000288644,ENSP00000502499,MAIMLLCLLQLAAPLCSYSITIRFYLFWLNTP
23778,NaN,ENSG00000288626,ENSP00000502794,MRDFPLAAGGTHPENAGAARGKDPLPQQRKTKRKKAYRQRALRDAP...
23779,NaN,ENSG00000288629,ENSP00000502602,MKTNTEMVCLRNFKRCRPPD


## NAN

In [86]:
print(clean_df.isna().sum())

Gene name         830
Gene stable ID      0
Protein_ID          0
protein_seq         0
dtype: int64


In [87]:
clean_df.to_csv("peptide_seq.csv",index= False)